In [1]:
from conch.open_clip_custom import create_model_from_pretrained, tokenize, get_tokenizer
import torch
import os
from PIL import Image
from pathlib import Path

# show all jupyter output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

c:\Users\Vivian\anaconda3\envs\conch2\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
root = Path('../').resolve()
os.chdir(root)

Load model from checkpoint

In [3]:
model_cfg = 'conch_ViT-B-16'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())  # Should return True if a GPU is available
print("PyTorch version:", torch.__version__)
print("CUDA version used by PyTorch:", torch.version.cuda)

# checkpoint_path = 'checkpoints/CONCH/pytorch_model.bin'
checkpoint_path = 'C:\\Users\\Vivian\\Documents\\CONCH\\checkpoints\\conch\\pytorch_model.bin' 
model, preprocess = create_model_from_pretrained(model_cfg, checkpoint_path, device=device)
_ = model.eval()

False
PyTorch version: 2.5.1+cpu
CUDA version used by PyTorch: None


c:\Users\Vivian\Documents\CONCH\conch\open_clip_custom\factory.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=map

In [4]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA version used by PyTorch:", torch.version.cuda)
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))


CUDA available: False
CUDA version used by PyTorch: None


AssertionError: Torch not compiled with CUDA enabled

In [5]:
import torch

print("Is CUDA available in PyTorch?", torch.cuda.is_available())
print("PyTorch version:", torch.__version__)
print("CUDA version compiled with PyTorch:", torch.version.cuda)
print("Number of GPUs detected:", torch.cuda.device_count())

if torch.cuda.is_available():
    print("Device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available in this PyTorch installation.")


Is CUDA available in PyTorch? False
PyTorch version: 2.5.1+cpu
CUDA version compiled with PyTorch: None
Number of GPUs detected: 0
CUDA is not available in this PyTorch installation.


Open an image and preprocess it

In [ ]:
# image = Image.open('C:\\Users\\Vivian\\Documents\\CONCH\\docs\\roi1.jpg')
# image = Image.open(r"C:\Users\Vivian\Documents\breakhis\BreaKHis_v1\BreaKHis_v1\histology_slides\breast\benign\SOB\adenosis\SOB_B_A_14-22549AB\40X\SOB_B_A-14-22549AB-40-001.png") # sample benign image from breakhis
image = Image.open(r"C:\Users\Vivian\Documents\breakhis\BreaKHis_v1\BreaKHis_v1\histology_slides\breast\malignant\SOB\ductal_carcinoma\SOB_M_DC_14-2523\40X\SOB_M_DC-14-2523-40-010.png") # sample malignant image from breakhis
# image = Image.open(r"C:\Users\Vivian\Documents\FA 57B-image.tif")
image_tensor = preprocess(image).unsqueeze(0).to(device)
image.resize((224, 224))

Load tokenizer and specify some prompts. Simplicity we just use one prompt per class (lung adenocarcinoma vs. lung squamous cell carcinoma) here instead ensembling multiple prompts / prompt templates.

In [ ]:
tokenizer = get_tokenizer()
classes = ['invasive ductal carcinoma', 
           'invasive lobular carcinoma']
prompts = ['an H&E image of invasive ductal carcinoma', 
           'an H&E image of invasive lobular carcinoma']

In [ ]:
tokenizer = get_tokenizer()
classes = ['benign', 
           'malignant']
prompts = ['an H&E image of benign breast tumor', 
           'an H&E image of malignant breast tumor']

In [ ]:
tokenized_prompts = tokenize(texts=prompts, tokenizer=tokenizer).to(device)
tokenized_prompts.shape

In [ ]:
with torch.inference_mode():
    image_embedings = model.encode_image(image_tensor)
    text_embedings = model.encode_text(tokenized_prompts)
    sim_scores = (image_embedings @ text_embedings.T * model.logit_scale.exp()).softmax(dim=-1).cpu().numpy()

print("Predicted class:", classes[sim_scores.argmax()])
print("Normalized similarity scores:", [f"{cls}: {score:.3f}" for cls, score in zip(classes, sim_scores[0])])